Action 1

In [ ]:
# directly copy this code and paste it in utils.py
def probability_of_valid_group_draw(players_hand, deck):
    # calculate the prob. of forming a valid group
    success_1 = 0
    success_2 = 0
    success_3 = 0
    success_4 = 0
    success_5 = 0
    success_6 = 0

    for first_card in range(len(deck.card)):
        hands_1 = copy.deepcopy(players_hand)
        copied_card = copy.deepcopy(deck.card[first_card])
        hands_1.append(copied_card)
        if find_possible_group(hands_1) != []:
            success_1 += 1
            success_2 += len(deck.card) -1
            success_4 += ((len(deck.card) - 1) * (len(deck.card)-2))
        for second_card in range(len(deck.card)):
            if second_card != first_card:
                hands_2 = copy.deepcopy(hands_1)
                copied_card = copy.deepcopy(deck.card[second_card])
                hands_2.append(copied_card)
                group = find_possible_group(hands_2)
                if group != []:
                    success_2 += 1
                    success_4 += len(deck.card) - 1
                for i in group:
                    for j in i:
                        first_not_in_group = True
                        second_not_in_group = True
                        if j.color == first_card.color and j.number == first_card.number:
                            first_not_in_group = False
                        elif j.color == second_card.color and j.number == second_card.number:
                          second_not_in_group = False
                    if first_not_in_group or second_not_in_group:
                        success_3 += 1
                        success_5 += len(deck.card) - 1
            for third_card in range(len(deck.card)):
                if third_card != first_card and third_card != second_card:
                    hands_3 = copy.deepcopy(hands_2)
                    copied_card = copy.deepcopy(deck.card[third_card])
                    hands_3.append(copied_card)
                    group = find_possible_group(hands_3)
                    if group != []:
                        success_4 += 1
                    for i in group:
                        for j in i:
                            first_not_in_group = True
                            second_not_in_group = True
                            third_not_in_group = True
                        if j.color == first_card.color and j.number == first_card.number:
                            first_not_in_group = False
                        elif j.color == second_card.color and j.number == second_card.number:
                          second_not_in_group = False
                        elif j.color == third_card.color and j.number == third_card.number:
                          third_not_in_group = False
                    if first_not_in_group or second_not_in_group or third_not_in_group:
                        success_5 += 1
                        success_6 += 1

  prob1 = success_1 / len(deck.card)
  prob2 = success_2 / math.perm(len(deck.card),2)
  prob3 = success_3 / math.perm(len(deck.card),2)
  prob4 = success_4 / math.perm(len(deck.card),3)
  prob5 = success_5 / math.perm(len(deck.card),3)
  prob6 = success_6 / math.perm(len(deck.card),3)

  return [prob1, prob2, prob3, prob4, prob5, prob6]

Action 2

In [ ]:
# directly copy this code and cover the original code in utils.py
import copy
def probability_of_valid_group_hand(players_hand, other_hand1, other_hand2 = None):
  # initialize variable
  success = 0
  prob = []

  #find prob. for 1st player
  for i in other_hand1:
      hands = copy.deepcopy(players_hand)
      hands.collection.append(i)
      if find_possible_groups(hands) != []:
          success += 1
  prob.append(success / len(other_hand1))

  #find prob. for 2nd player
  if other_hand2 is not None:
      success = 0
      for i in other_hand2:
          hands = copy.deepcopy(players_hand)
          hands.collection.append(i)
          if find_possible_groups(hands) != []:
              success += 1
      prob.append(success / len(other_hand2))

  return prob

Calling the function

In [ ]:
# directly copy this code and cover the original code in ai_player.py

from hand import *
from utils import *

class AIPlayer(Hand):
    def action1(self,num_to_draw):
        self.draw_cards(game_state['deck'], num_to_draw)
        self.actions_taken['draw'] = True

    def action2(self,target_player):
        self.take_card_from(target_player)
        self.actions_taken['steal'] = True

    def action3(self,group):
        largest_group = self.find_largest_valid_groups
        self.discard_group(largest_group, game_state['deck'])


class AggressiveAIPlayer(AIPlayer):
    def draw_decision(prob, hand):
        if len(hand) == 1:
            return 2
        elif len(hand) == 2:
            if hand[0].value == hand[1].value and hand[0].color != hand[1].color:
                return 1
            elif (hand[0].value == hand[1].value + 1 or hand[0].value == hand[1].value - 1)\
                 and hand[0].color == hand[1].color:
                 return 1
            else:
                num_to_draw = prob.index(max(prob)) + 1
        else:
            num_to_draw = prob.index(max(prob)) + 1
        return min(num_to_draw, 20-len(hand))


    def choose_action(self, game_state):
        # initializing variables
        other_players = game_state['players']
        groups = self.find_valid_groups()
        draw_prob = probability_of_valid_group_draw(self.cards,game_state['deck'])
        prob = [draw_prob[0], draw_prob[1], draw_prob[3]]
        num_to_draw = draw_decision(prob, self.cards)

        # making conditioins for actions
        if len(other_players) == 1:
            if len(other_players[0].cards) == 1:
                self.actions_taken['steal'] = True
                draw_condition = True
            else:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[0].cards)
                target_player = other_players[0]
                if draw_prob[num_to_draw - 1] >= steal_prob[0]:
                    draw_condition = True
                else:
                    draw_condition = False
        else:
            if len(other_players[0].cards) == 1 and len(other_players[1].cards) == 1:
                self.actions_taken['steal'] = True
                draw_condition = True
            elif len(other_players[0].cards) == 1:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[1].cards)
                target_player = other_players[1]
                if draw_prob[num_to_draw - 1] >= steal_prob[0]:
                    draw_condition = True
                else:
                    draw_condition = False
            elif len(other_players[1].cards) == 1:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[0].cards)
                target_player = other_players[0]
                if draw_prob[num_to_draw - 1] >= steal_prob[0]:
                    draw_condition = True
                else:
                    draw_condition = False
            else:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[0].cards, other_players[1].cards)
                if draw_prob[num_to_draw - 1] >= steal_prob[0] and draw_prob[num_to_draw - 1] >= steal_prob[1]:
                    draw_condition = True
                else:
                    draw_condition = False
                    if steal_prob[0] > steal_prob[1]:
                        target_player = other_players[0]
                    elif steal_prob[1] > steal_prob[0]:
                        target_player = other_players[1]
                    else:
                        target_player = max(other_players, key=lambda p: len(p.cards))

        # taking action
        if groups is not None:
            self.action3(group)
        elif (draw_condition or self.actions_taken['steal']) and not self.actions_taken['draw']:
            self.action1(num_to_draw)
        elif target_player is not None and not self.actions_taken['steal']:
            self.action2(target_player)

class DefensiveAIPlayer(AIPlayer):
    def draw_decision(prob, hand):
        if len(hand) == 1:
            return 2
        elif len(hand) == 2:
            if hand[0].value == hand[1].value and hand[0].color != hand[1].color:
                return 1
            elif (hand[0].value == hand[1].value + 1 or hand[0].value == hand[1].value - 1)\
                 and hand[0].color == hand[1].color:
                 return 1
            else:
                if prob[2] != 0:
                    num_to_draw = 3
                elif prob[1] != 0:
                    num_to_draw = 2
                else:
                    num_to_draw = 1
        else:
            if prob[2] != 0:
                num_to_draw = 3
            elif prob[1] != 0:
                num_to_draw = 2
            else:
                num_to_draw = 1
        return min(num_to_draw, 20-len(hand))


    def choose_action(self, game_state):
        # initializing variables
        other_players = game_state['players']
        groups = self.find_valid_groups()
        draw_prob = probability_of_valid_group_draw(self.cards,game_state['deck'])
        prob = [draw_prob[0], draw_prob[2], draw_prob[6]]
        num_to_draw = draw_decision(prob, self.cards)

        # making conditioins for actions
        if len(other_players) == 1:
            if len(other_players[0].cards) == 1:
                self.actions_taken['steal'] = True
                draw_condition = True
            else:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[0].cards)
                target_player = other_players[0]
                if draw_prob[num_to_draw - 1] >= steal_prob[0]:
                    draw_condition = True
                else:
                    draw_condition = False
                    if steal_prob[0] == 0:
                        target_player = None
        else:
            if len(other_players[0].cards) == 1 and len(other_players[1].cards) == 1:
                self.actions_taken['steal'] = True
                draw_condition = True
            elif len(other_players[0].cards) == 1:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[1].cards)
                target_player = other_players[1]
                if draw_prob[num_to_draw - 1] >= steal_prob[0]:
                    draw_condition = True
                else:
                    draw_condition = False
            elif len(other_players[1].cards) == 1:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[0].cards)
                target_player = other_players[0]
                if draw_prob[num_to_draw - 1] >= steal_prob[0]:
                    draw_condition = True
                else:
                    draw_condition = False
            else:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[0].cards, other_players[1].cards)
                if draw_prob[num_to_draw - 1] >= steal_prob[0] and draw_prob[num_to_draw - 1] >= steal_prob[1]:
                    draw_condition = True
                else:
                    draw_condition = False
                    if steal_prob[0] > steal_prob[1]:
                        target_player = other_players[0]
                    elif steal_prob[1] > steal_prob[0]:
                        target_player = other_players[1]
                    elif steal_prob[1] == 0 and steal_prob[0] == 0:
                        target_player = None
                    else:
                        target_player = max(other_players, key=lambda p: len(p.hand))

        # taking action
        if groups != []:
            self.action3(group)
        elif (draw_condition or self.actions_taken['steal']) and not self.actions_taken['draw']:
            self.action1(num_to_draw)
        elif target_player is not None and not self.actions_taken['steal']:
            self.action2(target_player)

class BalancedAIPlayer(AIPlayer):
    def draw_decision(prob, hand):
        if len(hand) == 1:
            return 2
        elif len(hand) == 2:
            if hand[0].value == hand[1].value and hand[0].color != hand[1].color:
                return 1
            elif (hand[0].value == hand[1].value + 1 or hand[0].value == hand[1].value - 1)\
                 and hand[0].color == hand[1].color:
                 return 1
            else:
                max_index = prob.index(max(prob))
                if max_index == 0:
                    num_to_draw = 1
                elif max_index == 1 or max_index ==2:
                    num_to_draw = 2
                else:
                    num_to_draw = 3
        else:
            max_index = prob.index(max(prob))
                if max_index == 0:
                    num_to_draw = 1
                elif max_index == 1 or max_index ==2:
                    num_to_draw = 2
                else:
                    num_to_draw = 3
        return min(num_to_draw, 20-len(hand))


    def choose_action(self, game_state):
        # initializing variables
        other_players = game_state['players']
        groups = self.find_valid_groups()
        draw_prob = probability_of_valid_group_draw(self.cards,game_state['deck'])
        num_to_draw = draw_decision(draw_prob, self.cards)

        # making conditioins for actions
        if len(other_players) == 1:
            if len(other_players[0].cards) == 1:
                self.actions_taken['steal'] = True
                draw_condition = True
            else:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[0].cards)
                target_player = other_players[0]
                if draw_prob[num_to_draw - 1] >= steal_prob[0]:
                    draw_condition = True
                else:
                    draw_condition = False
                    if steal_prob[0] == 0:
                        target_player = None
        else:
            if len(other_players[0].cards) == 1 and len(other_players[1].cards) == 1:
                self.actions_taken['steal'] = True
                draw_condition = True
            elif len(other_players[0].cards) == 1:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[1].cards)
                target_player = other_players[1]
                if draw_prob[num_to_draw - 1] >= steal_prob[0]:
                    draw_condition = True
                else:
                    draw_condition = False
            elif len(other_players[1].cards) == 1:
                steal_prob = probability_of_valid_group_hand(self.cards, other_players[0].cards)
                target_player = other_players[0]
                if draw_prob[num_to_draw - 1] >= steal_prob[0]:
                    draw_condition = True
                else:
                    draw_condition = False
            else:
                steal_prob = probability_of_valid_group_hand(self.hand, other_players[0].cards, other_players[1].cards)
                if draw_prob[num_to_draw - 1] >= steal_prob[0] and draw_prob[num_to_draw - 1] >= steal_prob[1]:
                    draw_condition = True
                else:
                    draw_condition = False
                    if steal_prob[0] > steal_prob[1]:
                        target_player = other_players[0]
                    elif steal_prob[1] > steal_prob[0]:
                        target_player = other_players[1]
                    elif steal_prob[1] == 0 and steal_prob[0] == 0:
                        target_player = None
                    else:
                        target_player = max(other_players, key=lambda p: len(p.hand))

        # taking action
        if groups != []:
            self.action3(group)
        elif (draw_condition or self.actions_taken['steal']) and not self.actions_taken['draw']:
            self.action1(num_to_draw)
        elif target_player is not None and not self.actions_taken['steal']:
            self.action2(target_player)